# OCR del periódico *El Martillo* (Chiclayo, 1903–1919)

Este cuaderno realiza el procesamiento digital de una página histórica del periódico peruano *El Martillo*. El objetivo es extraer el contenido mediante OCR usando la API de Claude (visión), organizar los textos en un formato estructurado y generar un pequeño análisis exploratorio.

## Pasos del proceso
1. Cargar la imagen escaneada de la página.
2. Enviar la imagen a Claude para realizar OCR.
3. Convertir la salida en un JSON válido y estructurado.
4. Construir un DataFrame y exportarlo como CSV.
5. Elaborar una visualización simple.


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from dotenv import load_dotenv
from anthropic import Anthropic
import base64

# Cargar variables de entorno (API key)
load_dotenv()

# Cliente de la API de Claude
client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

## Cargar la página escaneada
La ruta corresponde a la página elegida del periódico.

In [ ]:
image_path = "../data/el_martillo/page_01.png"
img = Image.open(image_path)
img

## Solicitud de OCR a Claude

Se pide a Claude que devuelva exclusivamente un JSON válido, sin comentarios ni explicaciones adicionales, para asegurar que pueda ser parseado sin problemas.

In [ ]:
prompt = """
Eres un asistente especializado en lectura y análisis de documentos históricos.
A partir de la imagen que recibirás, extrae todo el contenido legible del periódico.

Devuelve únicamente una lista en formato JSON. Cada elemento debe tener las
siguientes claves obligatorias:
- date
- issue_number
- headline
- section
- type (article / advertisement / other)
- text_excerpt

El JSON debe ser válido y parseable, sin texto adicional fuera del formato.
"""

with open(image_path, "rb") as f:
    image_bytes = f.read()

encoded_image = base64.b64encode(image_bytes).decode("utf-8")

response = client.messages.create(
    model="claude-3-5-sonnet-vision",
    max_tokens=4096,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": encoded_image
                    }
                }
            ]
        }
    ]
)

raw_output = response.content[0].text
raw_output

## Parseo del resultado en JSON
A veces Claude devuelve texto extra o saltos, así que incluimos un mecanismo para recuperar solo el bloque JSON, si fuera necesario.

In [ ]:
# Intentar cargar JSON directamente
try:
    structured = json.loads(raw_output)
except:
    # Intentar rescatar solo el bloque JSON
    import re
    matches = re.findall(r"\[.*\]", raw_output, re.DOTALL)
    if matches:
        structured = json.loads(matches[0])
    else:
        raise ValueError("Claude no devolvió un JSON válido.")

df = pd.DataFrame(structured)
df

## Exportar el CSV estructurado

In [ ]:
output_path = "../data/el_martillo/page_01_structured.csv"
df.to_csv(output_path, index=False)
df.head()

## Visualización simple
Se grafica la distribución de tipos de contenido (artículos, anuncios, otros).

In [ ]:
plt.figure(figsize=(6, 4))
df['type'].value_counts().plot(kind='bar')
plt.title('Distribución de tipos de contenido en la página')
plt.xlabel('Tipo')
plt.ylabel('Cantidad')
plt.show()